In [1]:
import sqlite3
import pandas as pd

In [2]:
from math import sqrt, atan2

In [3]:
NULL_SUBCARRIERS = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 64, 65, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127]

In [4]:
data = pd.read_csv("../training-data/Voxel-1/SittingDown-Movement-2024-04-11-13-50-48.csv")
data

,timestamp,rssi,csi,voxel_no,inter_voxel_no,activity
0,0,11,"[84, -64, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 14,...",1,5,SittingDown-Movement
1,0,11,"[84, -64, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, -15, 0...",1,5,SittingDown-Movement
2,0,11,"[84, -64, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 14, -4...",1,5,SittingDown-Movement
3,0,11,"[84, -64, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, -13, 7...",1,5,SittingDown-Movement
4,0,11,"[84, -64, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, -12, 9...",1,5,SittingDown-Movement
...,...,...,...,...,...,...
330,0,11,"[84, -64, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 13,...",1,5,SittingDown-Movement
331,0,11,"[84, -64, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, -5, -1...",1,5,SittingDown-Movement
332,0,11,"[84, -64, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 14...",1,5,SittingDown-Movement
333,0,11,"[84, -64, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 13,...",1,5,SittingDown-Movement


In [ ]:
# Remove the values in the array in the first column in CSI_DF whose indexes are given in the NULL_SUBCARRIERS array
# Convert the 'csi' column to a list of lists, assuming each entry is a string representation of a list
csi_df = data['csi'].apply(lambda x: eval(x))

# Define a function to remove null subcarriers
def remove_null_subcarriers(csi_list):
    return [value for index, value in enumerate(csi_list) if index not in NULL_SUBCARRIERS]

# Apply the function to each row in the 'csi' column
data['csi'] = csi_df.apply(remove_null_subcarriers)

In [ ]:
for index, row in data.iterrows():
    csi = row['csi']
    subcarrier_index = 0
    for i in range(len(csi)):
        if i % 2 == 0:
            subcarrier_index += 1
            imaginary = csi[i]
            real = csi[i+1]
            ampltiude = sqrt(imaginary ** 2 + real ** 2)
            phase = atan2(imaginary, real)
            data.loc[index, f'subcarrier_{subcarrier_index}_amplitude'] = ampltiude
            data.loc[index, f'subcarrier_{subcarrier_index}_phase'] = phase
data

In [ ]:
# Calculate the number of subcarriers
num_subcarriers = (len(data.columns) - 6) // 2  # Subtracting the original columns and dividing by 2 for amplitude and phase

# Create a list to store the vectors
vectors_list = []

# Calculate the vectors for each subcarrier and add them to the list
for subcarrier in range(1, num_subcarriers + 1):
    # Create column names for amplitude and phase
    amplitude_col_name = f'subcarrier_{subcarrier}_amplitude'
    phase_col_name = f'subcarrier_{subcarrier}_phase'
    
    # Calculate the average for each block of 67 rows for amplitude and phase
    amplitude_vector = data[amplitude_col_name].groupby(data.index // 67).mean().reset_index(drop=True)
    phase_vector = data[phase_col_name].groupby(data.index // 67).mean().reset_index(drop=True)
    
    # Add the vectors as a new row in the list
    vectors_list.append({
        'subcarrier': subcarrier,
        'amplitude_vector': amplitude_vector.tolist(),
        'phase_vector': phase_vector.tolist()
    })

# Convert the list of vectors into a DataFrame
vectors_df = pd.DataFrame(vectors_list)

# vectors_df now contains rows for each subcarrier with amplitude and phase vectors

In [2]:
connection = sqlite3.connect("../database/rti.db")

In [3]:
cursor = connection.cursor()

In [8]:
#Create table to save threshold of occurence for each activity for each voxel for each subcarrier
cursor.execute("""
    CREATE TABLE IF NOT EXISTS distance_vector (
        voxel TEXT,
        subcarrier INTEGER,
        activity TEXT,
        amplitude_vector,
        phase_vector
    )
""")

In [ ]:
voxel_identifier = 1
activity_name = 'SittingDown-Movement'

# Iterate over the DataFrame and insert each vector into the SQL table
for index, row in vectors_df.iterrows():
    subcarrier = row['subcarrier']
    amplitude_vector = str(row['amplitude_vector'])  # Convert list to string to store in SQL
    phase_vector = str(row['phase_vector'])  # Convert list to string to store in SQL
    
    # Insert the data into the distance_vector table
    cursor.execute("""
        INSERT INTO distance_vector (voxel, subcarrier, activity, amplitude_vector, phase_vector)
        VALUES (?, ?, ?, ?, ?)
    """, (voxel_identifier, subcarrier, activity_name, amplitude_vector, phase_vector))

# Commit the transaction
connection.commit()